In [1]:
import wikipedia
wikipedia.set_lang('ja')
p = None
# p = wikipedia.page('朝井リョウと加藤千恵のオールナイトニッポン0(ZERO)')
# p = wikipedia.page('ウレロ☆シリーズ')
p = wikipedia.page('怪奇恋愛作戦')
# p = wikipedia.page('アルコ&ピースのオールナイトニッポンシリーズ')
# p = wikipedia.page(pageid=129204)
# p = wikipedia.page('水曜どうでしょう')
# p = wikipedia.page('おぎやはぎのメガネびいき')
# p = wikipedia.page('リーガルハイ')
# p = wikipedia.page('リバースエッジ大川端探偵社')
# p = wikipedia.page('SPEC〜警視庁公安部公安第五課 未詳事件特別対策係事件簿〜')
# p = wikipedia.page('勝手にふるえてろ')
# p = wikipedia.page('ストロベリーナイト(テレビドラマ)')
# p = wikipedia.page('アイアンマン (映画)')
# print p.html()
# print p

In [2]:
import re
import numpy as np
import wikipedia
import bs4.element as bs4elem
from bs4 import BeautifulSoup

# todo handle English+漢字, '-.,' in the name
EXCEPTION = [u'A・T・C事務所',u'Scope co.,ltd.', u'4-Legs',  u'オア・グローリー神宮前店', u'インダストリアル・ライト&マジック']

# convert unicode to utf-8 if data is unicode
def utf(data):
    if isinstance(data, unicode):
        data = data.encode('utf-8')
    return data

# return true if data include only Katakana and alphabet
def isKatakanaOrEng(data):
    for d in data:
        if not ( '゠' <= utf(d) <= 'ヿ' or re.search('\w', utf(d)) ):
            return False
    
    return True

class WikiScraper(object):
    def __init__(self):
        self._name = ''
        self._page = ''
        self._bsObj = None
        self._headlines = []
        self._headline_tag = 'h3'
        self._last_key = ''
        self._result = dict()

    # return string of 'key : value \n'
    def __str__(self):
        output = self._name + '\n'
        for k, v in self._result.items():
            output +=  k + ' : ' 
            for staff in set(v):
                output += staff + ',       '
            output += u'\n'
        return output.encode('utf-8')
    
    # load html data from wikipedia
    # name: title of wikipage
    # lang: wikipedia language
    # pageid : id of wikipage.
    def load_wiki(self, name, lang='ja', pageid=np.nan):
        self._name = name
        wikipedia.set_lang(lang)
        try:
            if np.isnan(pageid):
                self._page = wikipedia.page(self._name)
            else:
                self._page = wikipedia.page(pageid=pageid)
            self._bsObj = BeautifulSoup(self._page.html(), "html.parser")   
        except:
            print('Can not open wikipage of '+self._name+' with lang '+lang)    
    
    # load html
    # name: title of wikipage
    # html: html string
    def load_html(self, name, html):
        self._bsObj = BeautifulSoup(html, "html.parser")   
        self._name = name
    
    # find headline
    # name name of headline, e.g. スタッフ
    def find_headline(self, name):
        headers = self._bsObj.find_all(class_='mw-headline')
        self._headlines = []
        for h in headers:
                if re.search(name, h.text):
                    self._headline_tag = h.parent.name
                    self._headlines.append(h)

    # cleauup text
    # wrapper function
    # remove brackets, blank, and \n
    def cleanup_text(func):
        def wrapper(self, key, data):
            value = []
            data = data.encode('utf-8')
            data = re.sub('(［|\[).+?(］|\])','', data) #remove brackets
            data = re.sub('(（|\(|\（).+?(）|\）|\))','', data) #remove brackets
            data = data.strip()  #remove \n, and blank from head and tail
            
            #exception
            for e in EXCEPTION:
                if e.encode('utf-8') in data:
                    value.append(e)
                    data = data.replace(e.encode('utf-8'),'')
            
            value = func(key, data, value)
            key = key.strip()
            if key in self._result:
                self._result[key].extend(value)
            else:
                 self._result[key] = value

        return wrapper

    # cleanup value
    # separate line with , / -> and etc. 
    @cleanup_text
    def cleanup_value_list(key, data, value):
        data = re.split('-', data)[0]
        
        data = re.split(u'[,、/／→\n]', data.decode('utf-8')) #separate multiple staffs    
        for d in data:
            d = d.strip()
            if not isKatakanaOrEng(d): #separate with '・' for ウレロシリーズ
                value.extend(re.split(u'[・]', d)) #separate multiple staffs    
#             value.extend(re.split(u'[,、/／→\n]', data.decode('utf-8'))) #separate multiple staffs    
            else:
                value.append(d) # do not separate with '・' for foregin people
#             value.extend(re.split(u'[,、/・／→\n]', data.decode('utf-8'))) #separate multiple staffs    

        for i, v in enumerate(value): #remove ※
            value[i]  = re.split(u'※|●', v)[0].strip()
        
        return value

    # cleanup value
    # separate line with , / -> and etc. 
    @cleanup_text
    def cleanup_value_infobox(key, data, value): 
        data = re.split(u'[※|●]', data)[0]
        data = re.split(u'[,、/→\n]', data.decode('utf-8')) #separate multiple staffs    
        for d in data:
            d = d.strip()
            if not isKatakanaOrEng(d): #separate with '・' for ウレロシリーズ
                value.extend(re.split(u'[・]', d)) #separate multiple staffs    
            else:
                value.append(d) # do not separate with '・' for foregin people

        return value
    
    # get list data
    def find_list(self, elt, parent=''):
        # recursive process if there are dl or ul
        for e in elt.find_all(['dl', 'ul']):
#             print e.parent.text
            if e.parent is not None:
                temp =  e.parent.text.split('\n')[0]
                if temp:
                    # researve text as parents
                    parent += e.parent.text.split('\n')[0] + ' '
                    self._last_key = parent
#             for s in e.parent.strings:
#                 parent += s.strip() + ' '
#                 self._last_key = parent
# #                 print parent
#                 break            
            self.find_list(e, parent)
            parent = ''
            if e.parent is not None:
                e.parent.decompose()
        
        # get data from each dd, li or dt
        for e in elt.find_all(['dd', 'li', 'dt']):
            data = e.text
            
            if e.name == 'dd':
                # add data if there is data already ref:朝井リョウ・加藤千恵のオールナイトニッポン staff
                if self._result.get(self._last_key) is not None:
                    data = self._result[self._last_key][-1]+ data
                key = self._last_key
                self._last_value = ''

            elif e.name == 'dt':
                # researve data as a parent
                parent = data
                self._last_key = parent
                continue
                
            elif not re.search(u'[:：-＝\-\=]',data): 
                # if data do not have :, - and etc, i.e. the separator of role and people, save data as a parent
                if parent == '':
                    parent = self._last_key
                key = parent.strip()
                self._last_value = ''

            else:
                #temp for 水曜どうでしょう　& アルピーANN
                if parent != '':
                    # if parents is not empty, set parent as key
                    key = parent
                    if re.search(u'：',data):
                        #if ':' is in data, split and data[0] is key and others will be data
                        datas = re.split(u'：', data)
                        key += datas[0]
                        
                        # connect data except for data[0] in case data has multiple ':'
                        data_temp = ''
                        for d in datas[1:]:
                            data_temp += d
                        data = data_temp
                    
                    elif re.search(u'-',data):
                        # separate data with '-'  and set first one as data, e.g. アルコ&ピースのANN
                        data = re.split(u'-', data)
                        key = parent
                        data = data[0]
                        
                else:
                    if re.search(u'[:：-＝\-\=]',data): 
                        # if parent is empty, separate data and set [0] as key and others as data
                        datas = re.split(u'[:：-＝\-\=]', data)
                        key = datas[0].strip() 
                        
                        data_temp = ''
                        for d in datas[1:]:
                            data_temp += d
                        data = data_temp

                    else :
                        #if parent is empty and no separator, use last key and
                        key = self._last_key
            
            value = self.cleanup_value_list(key, data)

            self._last_key = key
                
    def get_list_from_headline(self, name):
        self.find_headline(name)
        for headline_start in self._headlines:
            finish = False
            # get html between finded headline and next h[1-6]
            for elt in headline_start.parent.nextSiblingGenerator():
                if elt.name is not None and re.match('h[1-6]', elt.name) and elt.name <= self._headline_tag:
                    break

                if hasattr(elt, 'text'):
                    self.find_list(elt)

    def find_tr(self, table):
        rows = table.findAll('tr')
        for row in rows:
            if len(row.findAll('tr')):
                self.find_tr(row)
            else:
                index = row.find('th')
                if index is not None:
                    key = index.get_text()
                    value = row.find('td')
                    if value is not None:
                        # replace <br> with \n  to get by .text()
                        value_str = re.sub('</*br/*>', '\n', str(value))
                        value = BeautifulSoup(value_str, "html.parser")
        #                    value = value.get_text().split('\n')
                        value = self.cleanup_value_infobox(key, value.get_text())
        #                     self._result[key] = value


    def get_table(self, class_name='infobox'):
        tables = self._bsObj.findAll('table',{'class':class_name})
        for table in tables:
            self.find_tr(table)
                    
wsc = WikiScraper()
#wsc.load_wiki('朝井リョウと加藤千恵のオールナイトニッポン0(ZERO)', 'ja')
wsc.load_html(p.title, p.html())
# wsc.load_wiki(u'攻殻機動隊', lang='ja', pageid=129204)
wsc.get_list_from_headline(u'.*スタッフ')
wsc.get_table('infobox')
# print '-----------------------------'
print wsc

怪奇恋愛作戦
監督 : 白石和彌,       濱谷晃一,       ケラリーノ・サンドロヴィッチ,       
スタントコーディネーター : 高槻祐士,       
ジャンル : ラブコメディ,       
特殊メイクデザイン・造型 : 藤原カクセイ,       林屋陽二,       吉田茂正,       
オープニング : 女王蜂「ヴィーナス」,       
オープニングテーマ : 女王蜂「ヴィーナス」,       
放送国・地域 : 日本,       
エンディングタイトルロゴ&イラスト : ステレオテニス,       
ガンエフェクト : 会田文彦,       
プロデューサー : 濱谷晃一,       小松俊喜,       
チーフプロデューサー : 中川順平,       
回数 : 12,       
タイトルバック : 上田大樹,       
放送時間 : 土曜0:12 - 0:52,       
放送枠 : ドラマ24,       
撮影 : 神田創,       
製作著作 : 「怪奇恋愛作戦」製作委員会,       
コンテンツプロデューサー : 五箇公貴,       
放送期間 : 2015年1月10日 - 3月28日,       
エンディングアニメーション : 北村みなみ,       せきやすこ,       
脚本 : ケラリーノ・サンドロヴィッチ,       田中眞一,       小峯裕之,       関谷崇,       
音楽 : 鈴木慶一,       
助監督 : 鬼頭理三,       
製作総指揮 : 中川順平,       
エンディングテーマ : 電気グルーヴ「Fallin' Down」,       
出演者 : 緒川たまき,       麻生久美子,       坂井真紀,       仲村トオル,       
アシスタントプロデューサー : 小松幸敏,       
放送分 : 40åˆ†,       
エンディング : 電気グルーヴ「Fallin' Down」,       
照明 : 丸山和志,       
制作 : 楽映舎,       テレビ東京,       



In [3]:
import pandas as pd
data = pd.read_csv("contents.csv",index_col=0)
wscs = []
for i in data.index.values:
    wsc = WikiScraper()
    wsc.load_wiki(i.decode('utf-8'), pageid=data.at[i, 'id'], lang='ja')
#         wsc.load_wiki(i.decode('utf-8'), 'ja')
# #     wsc.load_html(p.html())
    if wsc._bsObj is not None:
        wsc.get_list_from_headline(u'.*スタッフ')
        wsc.get_table('infobox')
    
    wscs.append(wsc)
    
    print i
    print '-------------------------------------'
    print wsc
    print '+++++++++++++++++++++'

ウンナンさん
-------------------------------------
ウンナンさん
音声形式 : ステレオ放送,       
出演者 : 内村光良,       南原清隆,       内村宏幸,       
放送時間 : 火曜 23:55 - 翌0:35,       
放送枠 : Pooh!,       
放送分 : 40åˆ†,       
ジャンル : バラエティ番組,       
制作 : TBS,       
放送期間 : 2003年9月30日 - 2004年3月23日,       
放送国・地域 : 日本,       

+++++++++++++++++++++
ウンナンの気分は上々。
-------------------------------------
ウンナンの気分は上々。
音声形式 : ステレオ放送,       
音響効果 : 江本成治,       
監修 : 星野淳一郎,       
カメラ : 秋本新一,       
VTR編集 : 白澤淳,       反畑弘一,       村上健太郎,       田中直人,       松尾茂樹,       川中健治,       
演出協力 : ウイルスプロダクション,       
オープニング映像 : 小島淳二,       
公開放送 : 松元裕二,       
技術協力 : The TUBE,       東通,       TAMCO,       
制作協力 : 極東電視台,       ホリプロ,       
CAM : 秋本新一,       
ジャンル : バラエティ番組,       
衣裳 : 高野知子,       岩崎孝典,       
演出 : 中川通成,       安田淳,       杉本達,       田口健介,       ほか,       
チーフプロデューサー : 安田淳,       鶴岡滋之,       田代冬彦,       
コーディネーター : Top Taiwan,       
音効 : 江本成治,       
デスク : 椿美貴子,       
編成 : 坂田栄治,       
企画協力 : マセキ芸能社,       
美術プロデューサー : 山口智広,       小

マルさまぁ〜ず
-------------------------------------
マルさまぁ〜ず
音声形式 : モノラル放送,       
CAM : 松井光太郎,       
技術協力 : アンサーズ,       八峯テレビ,       
ジャンル : 深夜バラエティ番組,       
VE : 高山昌樹,       
編成担当 : 坂田栄治,       
音効 : 村松聡,       
TP : 田熊克二,       
構成 : 成瀬正人,       山内正之,       大井洋一,       福岡孝哲,       
宣伝 : 小林久幸,       
メイク : 田沢智美,       
プロデューサー : 飯沼美佐子,       
ナレーション : 大江吉史,       
演出 : 水野達也,       
AUD : 吉永哲也,       
CG : SPICIA,       
放送時間 : 毎週木曜 0:55 - 1:25,       
AP : 久田誠司,       
スタイリスト : 高村純子,       
放送期間 : 2010年4月8日 - 9月30日,       
放送国・地域 : 日本,       
MA : 天谷馬直男,       
出演者 : ,       さまぁ〜ず,       青木裕子,       
製作 : Gyama,       TBS,       
ディレクター : 塩谷泰孝,       
撮影協力 : マルハン,       
制作 : TBS,       
放送分 : 30åˆ†,       
エンディング : #エンディングテーマ参照,       
企画 : 大竹一樹,       
美術 : TACreate,       
編集 : 楠田惣彦,       

+++++++++++++++++++++
バナナ塾
-------------------------------------
バナナ塾
総合演出 : 有田武史,       
制作協力 : DCOMPLEX,       
CAM : 今井健一,       澤田基宏,       古俣智則,       
EED : 宮原明男,       木戸利彦,       
技術協力 : f

あらびき団
-------------------------------------
あらびき団
音声形式 : ステレオ放送,       
VTR編集 : 蓮田貴志,       
演出 : 原田浩司,       
総合演出 : 渡辺剛,       
ジャンル : バラエティ番組,       
PR : 國重希理佳,       
編成 : 嵯峨祥平,       山田康裕,       
衣装 : アマランス,       原口恵里,       
構成 : 興津豪乃,       清水隆佑,       佐藤篤志,       岸本浩二,       
メイク : 坂間亜由美,       
ナレーター : 服部伴蔵門,       
プロデューサー : 亀井俊徳,       中塚大悟,       村田聡子,       江間浩司,       宮本稔久,       
持道具 : 小松絵里子,       
AD : 由茅奈保美,       佐藤可菜里,       川島未幸,       田端裕一,       仲田和史,       濱田禮徳,       梅津菜々穂,       齊藤正明,       三浦匠,       石川千成,       田中雄大,       ナオミ・ヤユガ,       沖島新菜,       水上右太,       
小道具 : 川原栄一,       
AP : 土屋達彦,       中原隆二,       山下征志,       城下拓也,       沖本翔子,       八木愛子,       米村唯,       
装置操作 : 楢田祥之,       
放送時間 : 木曜日 2:10 - 3:40,       木曜日 23:56 - 翌0:55,       水曜日 23:59 - 翌0:29,       火曜日 23:50 - 翌0:20,       日曜日 2:50 - 3:50,       金曜日 23:50 - 翌3:50,       木曜日 0:10 - 1:40,       日曜日 1:10 - 2:40,       
放送期間 : 2016年12月28日,       2010年4月6日 - 2011年9月27日,       2017年7月13日,      

アフロの変
-------------------------------------
アフロの変
コンテンツデザイン : 阪本隆太,       柳田麻衣,       
音声形式 : ステレオ放送,       
制作協力 : UFIELD,       
CAM : 佐藤厚誠,       
技術協力 : imagica,       ニューテレス,       
ジャンル : バラエティ番組,       
AUD : 江川祐,       目野智子,       
美術協力 : フジアール,       
編成 : 村上正成,       藤井修,       
TP : 橋本雄司,       
放送期間 : 2014年10月17日- 2016年4月1日,       
LT : 安藤雄郎,       楢橋秀満,       
構成 : 林田晋一,       カツオ,       
TK : 岸田純子,       前場莉奈,       
制作著作 : フジテレビ,       
プロデューサー : 五十嵐剛,       下川猛,       高橋研,       上原敏明,       
ナレーション : 竹内友佳,       戸部洋子,       
演出 : 谷中憲,       
VE : 水野博道,       
キャスティング協力 : 柳橋弘紀,       
CG : 前川陽介,       
放送時間 : 金曜日 1:55 - 2:55,       
AP : 入江将也,       西村拓也,       大野もも,       
リサーチ : 川口夢斗,       鈴木悟史,       
チーフプロデューサー : 五十嵐剛,       下川猛,       
放送国・地域 : 日本,       
MA : 土屋信,       宝積伸示,       
出演者 : ほか,       大谷ノブ彦,       レキシ,       
制作 : フジテレビ　コンテンツ事業局,       フジテレビ,       総合開発局,       
ディレクター : 地濃愛,       武田雄一,       藤原将人,       原愼吾,       佐久間隆太,       
放送枠 : フジテレビからの!,    

リチャードホール
-------------------------------------
リチャードホール
作家 : 石原健次,       大平尚志,       渡辺真也,       宮藤官九郎,       藤谷弥生,       渡辺鐘,       福原フトシ,       松井洋介,       
プロデューサー : 清水宏泰,       
演出 : 伊藤征章,       
出演者 : 森三中,       劇団ひとり,       くりぃむしちゅー,       おぎやはぎ,       アンタッチャブル,       ほか,       中川家,       
ディレクター : 栗原真,       小仲正重,       
放送時間 : 水曜 24:35 - 24:58,       土曜 23:00 - 23:30,       
AP : 志賀直哉,       
放送枠 : 23 SATURDAY DREAM SHOW,       
放送分 : 30åˆ†,       23åˆ†,       
放送国・地域 : 日本,       
技術協力 : IMAGICA,       FLT,       ニユーテレス,       
ジャンル : お笑い番組,       
制作 : フジテレビ,       フジテレビバラエティ制作センター,       
放送期間 : 2005年4月9日 - 9月3日,       2004年10月20日 - 2005年3月,       
音声形式 : モノラル放送,       
制作著作 : フジテレビ,       

+++++++++++++++++++++
人生のパイセンtv
-------------------------------------
人生のパイセンtv
プロデューサー : 木月洋介,       中嶋優一,       
演出 : 萩原啓太,       
出演者 : オードリー,       
ディレクター : 飯田美沙紀,       
美術プロデューサー : 平岡慶大,       
放送時間 : 月曜日 0:30 - 1:00,       日曜日 23:15 - 23:45,       
放送期間 : 2015年10月18日 - 2016年3月20日,

...

ゴッドタン
-------------------------------------
ゴッドタン
監督 : 佐久間宣行,       
カメラ : 風間誠,       
演出補 : 岡千尋,       
録音 : 斉藤孝行,       松岡努,       永久保仁志,       
デザイン : 金森明日香,       
撮影監督 : 風間誠,       
美術進行 : 仙田拓也,       
制作協力 : 楽映舎,       TREX FILM,       シオプロ,       
監督・編集 : 佐久間宣行,       
ジャンル : バラエティ番組,       
技術協力 : テクノマックス,       
EED : 落合勇,       
言語 : 日本語,       
番宣 : 阪本裕美子,       
前作 : ゴッドタン キス我慢選手権 THE MOVIE,       
タイトル : 上田大樹,       
製作会社 : 「キス我慢選手権 THE MOVIE2」製作委員会,       「キス我慢選手権 THE MOVIE」製作委員会,       
映像 : 北村宏一,       小出一貴,       
衣装 : 杉本京加,       
技術統括 : 野瀬一成,       吉田健吾,       
構成 : 相澤昇,       永井ふわふわ,       オークラ,       大西右人,       成瀬正人,       くらなり,       
主題歌 : サンボマスター「孤独とランデブー」,       森山直太朗「五線譜を飛行機にして」,       
メイク : 蔵本優花,       綿谷紀子,       唐澤知子,       
プロデューサー / 演出 : 佐久間宣行,       
プロデューサー : 前田茂司,       五箇公貴,       佐久間宣行,       石井成臣,       露木寛子,       
ナレーション : 服部潤,       
美術・照明協力 : テレビ東京アート,       
演出 : 佐久間宣行,       
大道具 : 工藤義昭,       
公開 : 2014年10月17日,       2013年6月28日,       
小道具 

カウボーイビバップ
-------------------------------------
カウボーイビバップ
プロジェクト : アニメ,       
キャラクターデザイン : 川元利浩,       
監督 : 渡辺信一郎,       
監督・絵コンテ : 渡辺信一郎,       
アクション絵コンテ : 後藤雅巳,       中村豊,       出渕裕,       
文芸・設定制作 : 鳥羽聡,       
ポータル : アニメ,       
ジャンル : ハードボイルド,       SF,       スペースオペラ,       アクション,       
メカニカルデザイン : 山根公利,       
メカニック作画監督 : 後藤雅巳,       
舞台設定 : 佐藤大,       河森正治,       
封切日 : 2001年9月1日,       
ディスプレイデザイン : 佐山善則,       
放送局 : 超級電視台,       アダルトスイム,       WOWOW,       テレビ東京,       
キャラクターデザイン・作画監督 : 川元利浩,       
監督・構成 : 渡辺信一郎,       
プロデューサー : 南雅彦,       池口和彦,       
原作 : 矢立肇,       
演出 : 武井良幸,       
話数 : テレビ東京：全12話+総集編,       WOWOW：全26話+総集編,       
アニメーション制作 : サンライズ,       
音響監督 : 小林克良,       
設定協力 : 金子隆一,       今掛勇,       河森正治,       小林伸光,       
音響効果 : 倉橋静男,       
放送期間 : 1998年4月3日 - 6月26日,       テレビ東京：,       1998年10月23日 - 1999年4月23日,       WOWOW：,       
脚本 : 信本敬子,       
セットデザイン : 今掛勇,       竹内志保,       
メカニックデザイン : 山根公利,       
シリーズ構成 : 信本敬子,       
音楽 : 菅野よう子,       
製作 : バン

攻殻機動隊 STAND ALONE COMPLEX Solid State Society
-------------------------------------
攻殻機動隊 STAND ALONE COMPLEX Solid State Society
キャラクターデザイン : 西尾鉄也,       後藤隆幸,       
監督 : 神山健治,       
美術設定 : 渡部隆,       
撮影監督 : 田中宏待,       
配給 : Production I.G,       ティ・ジョイ,       
メカニカルデザイン : 寺岡賢司,       常木志伸,       
ジャンル : SFアニメ,       
言語 : 日本語,       
放送局 : パーフェクト・チョイス,       
3D監督 : 遠藤誠,       
美術監督 : 竹田悠介,       
プロデューサー : 石井朋彦,       
原作 : 士郎正宗,       
演出 : 河野利幸,       橘正紀,       吉原正行,       
アニメーション制作 : Production I.G,       
音楽監督 : 若林和弘,       
公開 : 2011年3月26日,       
放送期間 : 2006年9月1日 -,       
脚本 : 菅正太郎,       神山健治,       櫻井圭記,       
メカニックデザイン : 寺岡賢司,       常木志伸,       
音楽 : 菅野よう子,       
絵コンテ : 神山健治,       吉原正行,       
出演者 : 山口太郎,       大塚明夫,       田中敦子,       玉川紗己子,       榊原良子,       大川透,       阪脩,       山寺宏一,       仲野裕,       小野塚貴志,       
製作 : バンダイ エンタテインメント,       徳間書店,       BANDAI ENTERTAINMENT,       MANGA ENTERTAINMENT,       電通,       Production I.G,       バンダイビジュアル,       日本テレビ, 

スター・ウォーズ エピソード4/新たなる希望
-------------------------------------
スター・ウォーズ エピソード4/新たなる希望
監督 : ジョージ・ルーカス,       
特殊メイク : ダグ・ベズウィック,       リック・ベイカー,       スチュアート・フリーボーン,       
監督・脚本・製作総指揮 : ジョージ・ルーカス,       
言語 : 英語,       
吹替翻訳 : 大野隆一,       宇津木道子,       岡田壯平,       
日本語字幕 : 林完治,       岡枝慎二,       
製作会社 : ルーカスフィルム,       
製作費 : 000,       $11,       
配給 : 20世紀フォックス,       デジタル配信版,       ウォルト・ディズニー・スタジオ・モーション・ピクチャーズ,       オリジナル版＆特別篇,       
配給収入 : 43億8千万円,       
公開 : オリジナル版,       1978年6月24日,       1997年5月31日,       特別篇,       1977年5月25日,       1978年7月1日,       1997年1月31日,       
撮影 : ギルバート・テイラー,       
脚本 : ジョージ・ルーカス,       
音楽 : ジョン・ウィリアムズ,       
SFX : ILM,       ジョン・ダイクストラ,       リチャード・エドランド,       フィル・ティペット,       ジョン・スティアーズ,       
出演者 : アレック・ギネス,       ピーター・カッシング,       ハリソン・フォード,       キャリー・フィッシャー,       マーク・ハミル,       
製作 : 20世紀フォックス,       ゲイリー・カーツ,       ルーカスフィルム・リミテッド,       リック・マッカラム,       
製作総指揮 : ジョージ・ルーカス,       
上映時間 : 125分,       121分,       
興行収入 : 007,       $775,     

スター・ウォーズ/最後のジェダイ
-------------------------------------
スター・ウォーズ/最後のジェダイ
脚本 : ライアン・ジョンソン,       
原作 : キャラクター創造,       『スター,       ウォーズ』,       ジョージ・ルーカス,       
音楽 : ジョン・ウィリアムズ,       
前作 : フォースの覚醒,       スター・ウォーズ,       
監督 : ライアン・ジョンソン,       
製作 : ラム・バーグマン,       キャスリーン・ケネディ,       
製作会社 : ルーカスフィルム,       
製作総指揮 : J・J・エイブラムス,       ジェイソン・マクガトリン,       トム・カルノースキー,       
公開 : 2017年12月15日,       
製作費 : $200,       000,       000Â,       
興行収入 : 75.1億円,       889,       332,       539,       $1,       
撮影監督 : スティーヴ・イェドリン,       
言語 : 英語,       
配給 : ウォルト・ディズニー・スタジオ・モーション・ピクチャーズ,       
出演者 : ケリー・マリー・トラン,       ベニチオ・デル・トロ,       フランク・オズ,       ジョン・ボイエガ,       アダム・ドライバー,       ローラ・ダーン,       オスカー・アイザック,       キャリー・フィッシャー,       マーク・ハミル,       ドーナル・グリーソン,       デイジー・リドリー,       
撮影 : スティーヴ・イェドリン,       
日本語版スタッフ : 吹替版翻訳,       吹替版演出,       
製作国 : アメリカ合衆国,       
編集 : ボブ・ダクセイ,       
次作 : スター・ウォーズ,       スカイウォーカーの夜明け,       
上映時間 : 152åˆ†,       

+++++++++++++++++++++
ジャングルはいつもハレのちグゥ
-----

In [4]:
threshold = 1.0
max_number_of_nodes = 500
count_once = True

import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()

key_rm = [u'放送期間', u'放送時間', u'公開', u'上映時間', u'次作', u'回数', u'放送分']
value_rm = [u'', u'同上', u'日本', u'日本語', u'英語', u'公式サイト', u'ほか', u'ステレオ放送', u'文字多重放送',u'歴代エンディングテーマを参照',
            u'フジテレビ番組基本情報']

for wsc in wscs:
    
    point = float(data.at[wsc._name.encode('utf-8'), 'ポイント'])
    G.add_node(wsc._name, genre='content', point=point)
    attrs = set()
    for k, v in wsc._result.items():
        #add first genre to node attribute
        if k == u'ジャンル':
            if len(v)==0:
                G.node[wsc._name]['genre'] = v
            else:
                G.node[wsc._name]['genre'] = v[0]
        elif k not in key_rm:
            for attr in v:
                if attr not in data.index.values and attr not in value_rm: #do nothing for contents itself
                    if G.node.get(attr) is None: #first time
                        G.add_node(attr, genre='attribute', point=point)
                    elif not (attr in attrs and count_once): # from second time
                        G.node[attr]['point'] += point
                    G.add_edge(wsc._name, attr, relation=k) 
                    attrs.add(attr)
    print wsc._name, len(G.nodes)
        
cliques = nx.find_cliques(G)
# #標準出力
# for c in cliques:
#     print c



/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:28: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


ウンナンさん 6
ウンナンの気分は上々。 127
UN街 130
勇者ヨシヒコと魔王の城 131
勇者ヨシヒコと悪霊の鍵 174
勇者ヨシヒコと導かれし七人 188
ネリさまぁ〜ず 229
神さまぁ〜ず 253
ホリさまぁ〜ず 268
マルさまぁ〜ず 276
バナナ塾 318
バナナマンのブログ刑事 332
オトナ養成所_バナナスクール 354
ツギクルもん 405
うつけもん 420
オサレもん 426
30minutes 436
30minutes鬼 439
デリパンダ〜おしゃべりデリ坊、東京ド真ん中配達中〜 449
あらびき団 504
タイプライターズ 546
飛び出せ!科学くん 631
ラブレターズのオールナイトニッポン0(ZERO) 645
有田ジェネレーション 707
ワオ 750
アウトデラックス 810
アフロの変 854
未来ロケット 907
さんまのお笑い向上委員会 950
MASKMEN 973
タイノッチ 1004
主演 さまぁ〜ず 〜設定 美容室〜 1037
SENSORS 1126
リチャードホール 1142
人生のパイセンtv 1150
潜入捜査アイドル・刑事ダンス 1166
日曜×芸人 1190
Matthew's best hit TV 1263
どぅんつくぱ〜音楽の時間〜 1308
三四郎のオールナイトニッポン０ 1322
バナナマンのバナナムーンGold 1345
おぎやはぎのメガネびいき 1375
山里亮太の不毛な議論 1388
うしろシティ_星のギガボディ 1393
もろもろのハナシ 1459
怪奇恋愛作戦 1488
リバースエッジ大川端探偵社 1533
Creepy_Nutsのオールナイトニッポン0(ZERO) 1537
山田孝之の東京都北区赤羽 1569
わにとかげぎす 1605
スーパーサラリーマン左江内氏 1639
黒い十人の女 1698
She (テレビドラマ) 1705
おわこんTV 1735
バチバチエレキテる 1778
架空OL日記 1888
爆笑オンエアバトル 1892
考えるヒト 1907
LIFE!〜人生に捧げるコント〜 2053
小林賢太郎テレビ 2071
激レアさんを連れてきた 2227
万年B組ヒムケン先生 2269
クレイジージャーニー 2347
バナナ炎 2377
アオイホノオ 2429
アリ

In [5]:
# remove nodes for visualization
first = True
while len(G.nodes)>max_number_of_nodes or first:
    print 'Threshold: ', threshold
    print ' node num before simplify:', len(G.nodes)
    value_rm.extend([node for node,degree in dict(G.degree()).items() if degree < 2 and G.node[node]['genre'] is 'attribute']) #remove node based on degree
    value_rm.extend([node for node in G.nodes if  G.node[node]['point'] < threshold and G.node[node]['genre'] is 'attribute'])
    G.remove_nodes_from(value_rm)
    print ' node num after simplify:', len(G.nodes)
    threshold += 0.1
    first = False

#mix the closed genres
for v in data.index.values:
    title = v.decode('utf-8')
    if re.search( u'(.*バラエティ.*)|(.*お笑い.*)', G.node[title]['genre']):
        G.node[title]['genre'] = u'バラエティ'
    if re.search( u'.*ドラマ.*', G.node[title]['genre']):
        G.node[title]['genre'] = u'ドラマ'
    if re.search( u'.*SF.*', G.node[title]['genre']):
        G.node[title]['genre'] = u'SF'

nx.draw_networkx(G,font_family='AppleGothic',font_size =8)
# plt.show()
nx.write_gexf(G, 'result.gexf')

Threshold:  1.0
 node num before simplify: 6661
 node num after simplify: 1208
Threshold:  1.1
 node num before simplify: 1208
 node num after simplify: 1194
Threshold:  1.2
 node num before simplify: 1194
 node num after simplify: 1146
Threshold:  1.3
 node num before simplify: 1146
 node num after simplify: 1093
Threshold:  1.4
 node num before simplify: 1093
 node num after simplify: 1025
Threshold:  1.5
 node num before simplify: 1025
 node num after simplify: 915
Threshold:  1.6
 node num before simplify: 915
 node num after simplify: 785
Threshold:  1.7
 node num before simplify: 785
 node num after simplify: 705
Threshold:  1.8
 node num before simplify: 705
 node num after simplify: 652
Threshold:  1.9
 node num before simplify: 652
 node num after simplify: 634
Threshold:  2.0
 node num before simplify: 634
 node num after simplify: 598
Threshold:  2.1
 node num before simplify: 598
 node num after simplify: 588
Threshold:  2.2
 node num before simplify: 588
 node num after si

In [8]:
#option1
#visualize with pyvis + networkx 
from pyvis.network import Network
# nxg = nx.complete_graph(10)
# gg = Network(notebook=True)
# gg.from_nx(G)
# gg.show("nx.html")

def get_net_node(net, key, value):
    for item in net.nodes:
        if item[key] == value:
            return item
    return None 

net = Network(height='500px', width='1000px')
# net = Network(notebook=True)
net.force_atlas_2based(
    gravity=-30, 
    central_gravity=0.01, 
    spring_length=1.0, 
    spring_strength=0.8, 
    damping=1.0, 
    overlap=0.5)

node_id = 0
for label in G.nodes:
#     print label, G.node[label]['genre'], G.node[label]['point']
    if G.node[label]['genre'] == 'attribute':
        shape = 'dot'
    else:
        shape = 'box'
    net.add_node(node_id, label=label, 
                 group=G.node[label]['genre'], 
                 value=G.node[label]['point']*1.0,
                 mass=G.node[label]['point'],
                shape=shape)
    node_id += 1
    
for edge in G.edges:
    edge[0], edge[1]
    net.add_edge( get_net_node(net, 'label', edge[0])['id'], 
                 get_net_node(net, 'label', edge[1])['id'])

net.show_buttons(filter_=['physics'])
net.save_graph("result.html")

In [ ]:
#opition2
#visualize with pyvis only
from pyvis.network import Network
def get_net_node(net, key, value):
    for item in net.nodes:
        if item[key] == value:
            return item
    return None 

key_rm = [u'放送期間', u'放送時間', u'公開', u'上映時間', u'次作', u'回数', u'放送分']
value_rm = [u'', u'同上', u'日本', u'日本語', u'英語', u'公式サイト', u'ほか', u'ステレオ放送', u'文字多重放送',u'歴代エンディングテーマを参照',
            u'フジテレビ番組基本情報']

net = Network(height='500px', width='1000px')
# net = Network(notebook=True)

count_once = True
node_id = 0
for wsc in wscs:
    point = float(data.at[wsc._name.encode('utf-8'), 'ポイント'])    
    net.add_node(node_id, label=wsc._name, group='content', value=point)
    node_id += 1
    attrs = set()
    #print wsc._name, point
    genre = ''
    for k, v in wsc._result.items():
        #add first genre to node attribute
        if k == u'ジャンル':
            if len(v)==0:
                genre = v
            else:
                genre = v[0]
            get_net_node(net, 'label', wsc._name)['group'] = genre
        elif k not in key_rm:
            for attr in v:
                if attr not in data.index.values and attr not in value_rm: #do nothing for contents itself and value_rm
                    if get_net_node(net, 'label', attr) is None: #first time
                        net.add_node(node_id, label=attr, group='attribute', value=point)
                        node_id += 1

                    elif not (attr in attrs and count_once): # from second time in the same page
                        get_net_node(net, 'label', attr)['value'] += point
                    
                    net.add_edge( get_net_node(net, 'label', wsc._name)['id'], get_net_node(net, 'label', attr)['id'])
                    attrs.add(attr)
    print wsc._name, len(net.nodes)

In [ ]:
# remove nodes for visualization
threshold = 1.0
max_number_of_nodes = 500
count_once = True
first = True
while len(net.nodes)>max_number_of_nodes or first:
    print 'Threshold: ', threshold
    print ' node num before simplify:',len(net.nodes)
    net.nodes = [node for node in net.nodes 
                 if (node['value'] > threshold and node['group'] == 'attribute') 
                 or node['group'] != 'attribute']
    print ' node num after simplify:', len(net.nodes)
    threshold += 0.1
    first = False

#mix the closed genres
for v in data.index.values:
    title = v.decode('utf-8')
#     print title, get_net_node(net, 'label', title)
    if re.search( u'(.*バラエティ.*)|(.*お笑い.*)', get_net_node(net, 'label', title)['group']):
        get_net_node(net, 'label', title)['group'] = u'バラエティ'
    if re.search( u'.*ドラマ.*', get_net_node(net, 'label', title)['group']):
        get_net_node(net, 'label', title)['group'] = u'ドラマ'
    if re.search( u'.*SF.*', get_net_node(net, 'label', title)['group']):
        get_net_node(net, 'label', title)['group'] = u'SF'

net.save_graph("test.html")
net.show("test.html")